Setup Scripts

# **Wrap Scripts**

In [ ]:

! git clone https://github.com/salesforce/TabularSemanticParsing
%cd TabularSemanticParsing

! pip install torch torchvision
! python3 -m pip install -r requirements.txt
! pip install flask-ngrok
! pip install flask==0.12.2

! export PYTHONPATH=`pwd` && python -m nltk.downloader punkt

! mkdir model

! gdown https://drive.google.com/uc?id=1dlrUdGMLvvvfR3kNVy76H12rR7gr4DXI
! mv bridge-spider-bert-large-ems-70-1-exe-68-2.tar.gz model
! gunzip model/bridge-spider-bert-large-ems-70-1-exe-68-2.tar.gz

! gdown https://drive.google.com/uc?id=1vk14iR4V_f5x4e17MAaL_L8T9wgjcKCy
! mv dev.eo.pred.restored.pkl.gz data/spider
! gunzip data/spider/dev.eo.pred.restored.pkl.gz

Cloning into 'TabularSemanticParsing'...
remote: Enumerating objects: 422, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 422 (delta 0), reused 0 (delta 0), pack-reused 419
Receiving objects: 100% (422/422), 1.12 MiB | 17.19 MiB/s, done.
Resolving deltas: 100% (224/224), done.
/content/TabularSemanticParsing
     |████████████████████████████████| 44 kB 1.9 MB/s 
     |████████████████████████████████| 54 kB 3.4 MB/s 
     |████████████████████████████████| 59 kB 8.0 MB/s 
     |████████████████████████████████| 1.4 MB 68.1 MB/s 
     |████████████████████████████████| 12.9 MB 31.9 MB/s 
     |████████████████████████████████| 563 kB 36.5 MB/s 
     |████████████████████████████████| 1.5 MB 60.8 MB/s 
     |████████████████████████████████| 2.0 MB 51.1 MB/s 
     |████████████████████████████████| 1.3 MB 52.4 MB/s 
     |████████████████████████████████| 180 kB 52.7 MB/s 
     |████████████████████████████████| 140 kB 73.5

Upload chinook database

In [ ]:
! gdown https://drive.google.com/uc?id=1_AckYkinAnhqmRQtGsQgUKAnTHxxX5J0
! unzip spider.zip -d .

Downloading...
From: https://drive.google.com/uc?id=1_AckYkinAnhqmRQtGsQgUKAnTHxxX5J0
To: /content/TabularSemanticParsing/spider.zip
100% 99.7M/99.7M [00:01<00:00, 61.7MB/s]
Archive:  spider.zip
   creating: ./spider/
   creating: ./spider/database/
   creating: ./spider/database/customer_deliveries/
  inflating: ./spider/database/customer_deliveries/schema.sql  
  inflating: ./spider/database/customer_deliveries/customer_deliveries.sqlite  
   creating: ./spider/database/allergy_1/
  inflating: ./spider/database/allergy_1/schema.sql  
  inflating: ./spider/database/allergy_1/allergy_1.sqlite  
   creating: ./spider/database/company_office/
  inflating: ./spider/database/company_office/schema.sql  
  inflating: ./spider/database/company_office/company_office.sqlite  
   creating: ./spider/database/device/
  inflating: ./spider/database/device/schema.sql  
  inflating: ./spider/database/device/device.sqlite  
   creating: ./spider/database/phone_1/
  inflating: ./spider/database/phone_1

In [ ]:
! cp -a spider/. data/spider

# **DB Setup**

In [ ]:
import json

def merge_two_objects(path1, path2, out_path):
    with open(path1, 'r') as file:
      f1 = json.load(file)
    
    with open(path2, 'r') as file:
      f2 = json.load(file)
     
    f1.extend(f2)

    with open(out_path, 'w') as file:
      json.dump(f1, file)

merge_two_objects('/content/TabularSemanticParsing/data/spider/train_others.json', 
                  '/content/TabularSemanticParsing/data/spider/train_spider.json',
                  '/content/TabularSemanticParsing/data/spider/train.json')

# **Inference stuff**

In [ ]:
%cd TabularSemanticParsing
import sqlalchemy
from sqlalchemy import MetaData
from src.data_processor.schema_graph import SchemaGraph
from src.demos.demos import Text2SQLWrapper

[Errno 2] No such file or directory: 'TabularSemanticParsing'
/content/TabularSemanticParsing


In [ ]:
db = sqlalchemy.create_engine('sqlite:////content/TabularSemanticParsing/chinook.db')

In [ ]:
class NaturalQuery(object):
    def __init__(self, db, name):
      super().__init__()

      self.db = db
      
      metadata = MetaData(db)
      metadata.reflect()

      self.metadata = metadata

      schema = SchemaGraph(name=name)
      schema.load_data_from_sqlalchemy_metadata(self.metadata.tables.values())

      self.schema = schema

    def args():
      # I need: path_to_checkpoint
      pass

In [ ]:
from src.trans_checker.args import args as cs_args

In [ ]:
class Args(object):
    def __init__(self):
        self.model = 'bridge'
        self.model_id = 2
        self.inference = True
        self.checkpoint_path='/content/TabularSemanticParsing/model/bridge-spider-bert-large-ems-70-1-exe-68-2.tar'
        self.model_dir = 'model'
        self.start_step = 0
        self.num_log_steps = 500
        self.optimizer = 'adam'
        self.adam_beta1 = 0.9
        self.adam_beta2 = 0.999
        self.adam_eps = 1e-8
        self.save_all_checkpoints = False
        self.viz_dir = '/content/TabularSemanticParsing/viz'
        self.xavier_initialization = True
        self.decoder_hidden_dim = -1
        self.ff_output_dropout_rate = 0.0
        self.loss = "cross_entropy"
        self.train = False
        self.test = False

    data_dir="data/spider"
    db_dir="data/spider/database"
    dataset_name="spider"
    model="bridge"
    question_split=True
    query_split=False
    question_only=True
    normalize_variables=False
    denormalize_sql=True
    omit_from_clause=False
    no_join_condition=False
    table_shuffling=True
    use_lstm_encoder=True
    use_meta_data_encoding=True
    use_graph_encoding=False
    use_typed_field_markers=False
    use_picklist=False
    read_picklist=False
    anchor_text_match_threshold=0.85
    no_anchor_text=False
    top_k_picklist_matches=2
    sql_consistency_check=True
    atomic_value_copy=False
    process_sql_in_execution_order=True
    share_vocab=False
    sample_ground_truth=False
    save_nn_weights_for_visualizations=False
    vocab_min_freq=0
    text_vocab_min_freq=0
    program_vocab_min_freq=0
    max_in_seq_len=512
    max_out_seq_len=60

    num_steps=100000
    curriculum_interval=0
    num_peek_steps=1000
    num_accumulation_steps=2
    save_best_model_only=True
    train_batch_size=16
    dev_batch_size=24
    encoder_input_dim=1024
    encoder_hidden_dim=400
    decoder_input_dim=400
    num_rnn_layers=1
    num_const_attn_layers=0

    use_oracle_tables=False
    num_random_tables_added=0
    use_additive_features=False

    schema_augmentation_factor=1
    random_field_order=False
    data_augmentation_factor=1
    augment_with_wikisql=False
    num_values_per_field=0
    pretrained_transformer="bert-large-uncased"
    fix_pretrained_transformer_parameters=False
    bert_finetune_rate=0.00006
    learning_rate=0.0005
    learning_rate_scheduler="inverse-square"
    trans_learning_rate_scheduler="inverse-square"
    warmup_init_lr=0.0005
    warmup_init_ft_lr=0.00003
    num_warmup_steps=4000
    emb_dropout_rate=0.3
    pretrained_lm_dropout_rate=0
    rnn_layer_dropout_rate=0
    rnn_weight_dropout_rate=0
    cross_attn_dropout_rate=0
    cross_attn_num_heads=8
    res_input_dropout_rate=0.2
    res_layer_dropout_rate=0
    ff_input_dropout_rate=0.4
    ff_hidden_dropout_rate=0.0

    grad_norm=0.3
    decoding_algorithm="beam-search"
    beam_size=16
    bs_alpha=1.05

    data_parallel=False

    leaderboard_submission=False
    demo=False
    use_pred_tables = False
    execution_guided_decoding = False

In [ ]:
#metadata = MetaData(db)
#metadata.reflect()

#schema = SchemaGraph(name='chinoo')
#schema.load_data_from_sqlalchemy_metadata(metadata.tables.values())

In [ ]:
def create_t2s_model(args, cs_args, schema):
  return Text2SQLWrapper(args, cs_args, schema)

In [ ]:
schema = SchemaGraph(name='default')
args = Args()
t2s = create_t2s_model(args, cs_args, schema)

* text vocab size = 30522
* program vocab size = 99




=> no checkpoint found at '/content/TabularSemanticParsing/model/trans_check_0.0001_3e-05_800/model-best.tar'
./content created
./content/TabularSemanticParsing created
./content/TabularSemanticParsing/viz created
* text vocab size = 30522
* program vocab size = 99

pretrained_transformer = bert-large-uncased
fix_pretrained_transformer_parameters = False

bridge module created
=> loading checkpoint '/content/TabularSemanticParsing/model/bridge-spider-bert-large-ems-70-1-exe-68-2.tar'


In [ ]:
! pip install flask-cors

In [ ]:
# flask_ngrok_example.py
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from flask_cors import CORS

app = Flask(__name__)
CORS(app)
run_with_ngrok(app)  # Start ngrok when app is run
app.config['JSONIFY_PRETTYPRINT_REGULAR'] = False

@app.route("/")
def hello():
    return "Hello World!"


@app.route("/schema", methods=['POST'])
def set_schema():
    schema_data = request.get_json(force=True)
    name = schema_data['name']
    new_schema = SchemaGraph(name=name)
    print(schema_data)
    new_schema.load_data_from_spider_json(schema_data)
    t2s.add_schema(new_schema)
    return 'ok'


@app.route("/query", methods=['POST'])
def query():
    data = request.get_json(force=True)
    print(data)
    name = data['name']
    text = data['text']
    answer = t2s.process(text, name)
    return jsonify(answer)

if __name__ == '__main__':
    app.run()  # If address is in use, may need to terminate other sessions:
               # Runtime > Manage Sessions > Terminate Other Sessions

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://ff13-35-203-150-144.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [23/Dec/2021 09:00:24] "OPTIONS //schema HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2021 09:18:48] "OPTIONS //schema HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2021 09:19:48] "OPTIONS //query HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2021 09:19:56] "OPTIONS //query HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2021 09:20:31] "OPTIONS //query HTTP/1.1" 200 -
